In [1]:
pip install torch


In [2]:
pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.6 MB/s eta 0:00:00


In [3]:
pip install torch-geometric==2.4.0 torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.5.0+cu121.html


Looking in links: https://data.pyg.org/whl/torch-2.5.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: torch-geometric
    Found existing installation: torch-geometric 2.7.0
    Uninstalling torch-geometric-2.7.0:
      Successfully uninstalled torch-geometric-2.7.0


In [6]:


import networkx as nx
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),
         (3,4), (4,5), (5,3)]
G.add_edges_from(edges)

# One-hot node features
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Masks
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

# -------------------------------------------
# Define a simple 2-layer GCN
# -------------------------------------------
class BotGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(num_nodes, 4)
        self.conv2 = GCNConv(4, 2)  # 2 classes (normal/bot)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Create model
model = BotGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# -------------------------------------------
# Training loop
# -------------------------------------------
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# -------------------------------------------
# Evaluation
# -------------------------------------------
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()

print("Predictions:", pred.tolist())
print(f"Test Accuracy: {acc:.2f}")


Predictions: [0, 0, 0, 1, 1, 1]
Test Accuracy: 1.00
